In [1]:
import sys
sys.path.insert(0, "../../src")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from gen.util import read_data, write_jsonl
from rte.aggregate import generate_micro_macro_df

In [2]:
root_data = Path("../../data").resolve()
root_model = Path("../../models").resolve()

In [3]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [4]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)
import torch
torch.backends.cuda.matmul.allow_tf32 = True

/scratch/users/k21190024/envs/conda/mainenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Huggingface Init

## Model

In [5]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [6]:
model_checkpoint = "bert-base-uncased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"], max_length=512, truncation="only_first")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
dataset = ["fever", "climatefeverpure", "fever-climatefeverpure"]
doc_sent = ["doc", "sent"]

di = 2
ds = 0

model_store_path = root_model.joinpath("sentence-models" if ds == 1 else "document-models")
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}"

## Dataset

In [8]:
datap = root_data / f"{doc_sent[ds]}-dataset"

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "fever_test": Dataset.from_list(read_data(datap / f"{dataset[0]}.test.n5.jsonl")),
    "climatefever_test": Dataset.from_list(read_data(datap / f"{dataset[1]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

## Trainer

In [11]:
batch_size = 32
learning_rate = 2e-5
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [5e-5, 3e-5, 2e-5]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=10, 
    hp_space=optuna_hp_space
)

[I 2023-07-04 13:44:38,036] A new study created in memory with name: no-name-22429e36-ada3-4df8-bb9e-33d8cc219dda
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.590838,0.653693,0.649690,0.824519,0.544652
400,No log,0.315366,0.892284,0.891552,0.900270,0.891459
600,0.434000,0.278977,0.899290,0.898646,0.905239,0.898105
800,0.434000,0.281363,0.903766,0.902989,0.914432,0.903070
1000,0.212600,0.318529,0.903960,0.903366,0.909012,0.903025
1200,0.212600,0.249680,0.916513,0.915994,0.921238,0.916243
1400,0.212600,0.271009,0.911647,0.910966,0.919599,0.911099
1600,0.153300,0.246249,0.922156,0.921712,0.925279,0.921676
1800,0.153300,0.220190,0.924394,0.924152,0.925721,0.924473
2000,0.125400,0.314358,0.921378,0.920906,0.925193,0.921115


[I 2023-07-04 13:57:21,457] Trial 0 finished with value: 3.7116962953643977 and parameters: {'learning_rate': 5e-05, 'num_train_epochs': 5}. Best is trial 0 with value: 3.7116962953643977.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (in

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.667036,0.651163,0.647167,0.486575,0.539921
400,No log,0.494824,0.794493,0.792380,0.838563,0.779226
600,0.528100,0.267156,0.898900,0.898392,0.901689,0.898118
800,0.528100,0.275070,0.905517,0.904841,0.913710,0.905153
1000,0.207900,0.302457,0.906490,0.905840,0.911936,0.905389
1200,0.207900,0.253134,0.918264,0.917737,0.923638,0.918249
1400,0.207900,0.241218,0.918167,0.917681,0.922086,0.917947
1600,0.155900,0.245108,0.917778,0.917259,0.922570,0.917654
1800,0.155900,0.211130,0.928968,0.928583,0.932085,0.928981
2000,0.130800,0.255262,0.928189,0.927842,0.930780,0.928281


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-04 14:10:01,960] Trial 1 finished with value: 3.7166557780025076 and parameters: {'learning_rate': 2e-05, 'num_train_epochs': 5}. Best is trial 1 with value: 3.7166557780025076.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassi

...

[I 2023-07-04 14:50:24,871] Trial 7 pruned. 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.648853,0.652039,0.648028,0.489253,0.541232


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-04 14:50:58,875] Trial 8 pruned. 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification 

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.442202,0.799455,0.797848,0.821781,0.790572
400,No log,0.327025,0.886445,0.885522,0.898133,0.884845
600,0.396700,0.311826,0.885667,0.884742,0.899731,0.884841


[I 2023-07-04 14:52:39,375] Trial 9 pruned. 


In [13]:
best_run

BestRun(run_id='2', objective=3.731241053209169, hyperparameters={'learning_rate': 5e-05, 'num_train_epochs': 4}, run_summary=None)

## Train with best hyperparameters

In [ ]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=1000,
    save_strategy = "steps",
    save_steps=1000,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.218700,0.236582,0.918751,0.918158,0.925179,0.918469
2000,0.178500,0.212528,0.932860,0.932448,0.935645,0.932553
3000,0.173100,0.182076,0.937530,0.937162,0.940182,0.937454
4000,0.167300,0.194541,0.941909,0.941669,0.943992,0.942170
5000,0.122200,0.197473,0.942006,0.941741,0.943291,0.941942
6000,0.122800,0.178773,0.944244,0.943941,0.946652,0.944370
7000,0.109800,0.197495,0.940158,0.939830,0.942155,0.940072
8000,0.117700,0.183740,0.944244,0.943902,0.946076,0.943899
9000,0.112500,0.170650,0.949693,0.949527,0.950453,0.949768
10000,0.069800,0.234957,0.943077,0.942854,0.944713,0.943282


In [29]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [22]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [23]:
preds = trainer.predict(data["validation"])
val = generate_doc_df(data["validation"], preds)

In [24]:
print(classification_report(y_true=val["actual"], y_pred=val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      0.99      0.99      3428
        REFUTES       0.95      0.91      0.93      3384
       SUPPORTS       0.91      0.96      0.94      3465

       accuracy                           0.95     10277
      macro avg       0.95      0.95      0.95     10277
   weighted avg       0.95      0.95      0.95     10277



## Test on test data

### FEVER

In [25]:
preds = trainer.predict(data["fever_test"])
fval = generate_doc_df(data["fever_test"], preds)

In [26]:
print(classification_report(y_true=fval["actual"], y_pred=fval["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      0.99      1.00      3333
        REFUTES       0.95      0.88      0.91      3333
       SUPPORTS       0.89      0.96      0.92      3333

       accuracy                           0.94      9999
      macro avg       0.95      0.94      0.94      9999
   weighted avg       0.95      0.94      0.94      9999



### Climate-FEVER

In [27]:
preds = trainer.predict(data["climatefever_test"])
cfval = generate_doc_df(data["climatefever_test"], preds)

In [28]:
print(classification_report(y_true=cfval["actual"], y_pred=cfval["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.91      0.85      0.88        47
        REFUTES       0.46      0.44      0.45        25
       SUPPORTS       0.74      0.78      0.76        65

       accuracy                           0.74       137
      macro avg       0.70      0.69      0.70       137
   weighted avg       0.75      0.74      0.74       137

